In [1]:
# this file takes in all files and produce the appropriate d2s analysis
from pathlib import Path
from datasets import load_dataset
from evaluate import load
import pandas as pd
import numpy as np
from pathlib import Path
from functional import seq
from funcutils import underscore as _
from funcutils import get
from IPython.display import display, display_html, HTML
from editdistance import distance as edit_distance

import matplotlib.pyplot as plt

In [2]:
plt.style.use('seaborn-v0_8-whitegrid')
params = {"ytick.color" : "black",
          "xtick.color" : "black",
          "axes.labelcolor" : "black",
          "axes.edgecolor" : "black",
          "text.usetex" : True,
          "font.family" : "serif",
          "font.serif" : ["Computer Modern Serif"]}
plt.rcParams.update(params)

In [3]:
dspl_html = lambda x: display_html(x, raw=True)
rouge = load('rouge')

In [4]:
root_path = Path("/home/vente/repos/nlgs-research")


INPUT_TO_ANALYSE = ""

# TODO: get this file form argv so driver code can run all analyses in a loop
# or just analayse the outputs of the best model...?
pkl = max( (root_path / "pipeline/predictions").glob("*d2s*"))
pkl.name

'd2s-t5-small-5.pkl'

In [5]:
OUTPUT_PATH = root_path / "pipeline/scores" / pkl.name.removesuffix(".pkl")
OUTPUT_PATH.mkdir(exist_ok=True)
OUTPUT_PATH

PosixPath('/home/vente/repos/nlgs-research/pipeline/scores/d2s-t5-small-5')

In [6]:
test_predictions = pd.read_pickle(pkl)

In [7]:
compute_rouge = lambda x,y: rouge.compute(references=[x], predictions=[y], use_stemmer=False, use_aggregator=False)
compute_rouge(["general kenobi"], "general kenobi")
y_pred = test_predictions.drop(columns=['input_ids','attention_mask','pred_ids','labels'])

chunked = (
  seq(y_pred.to_dict('records'))
    .group_by(get.record_idx)
    .map(get[1]) # focus on teh values
    .map(lambda x: [
      seq(x).map(get.nl).to_list(),        # gather up all of the references
      seq(x).map(get.decoded).to_list()[0] # and the predictions
    ])
)
chunked

"['Abilene, Texas is served by the Abilene regional airport.', 'Abilene Regional Airport serves the city of Abilene in Texas.']","Abilene Regional Airport serves the city of Abilene, Texas."
"['Adolfo Suarez Madrid-Barajas Airport can be found in Madrid, Paracuellos de Jarama, San Sebastian de los Reyes and Alcobendas.', 'Adolfo Suarez Madrid-Barajas airport is located at Madrid, Paracuellos de Jarama, San Sebastian de los Reyes and Alcobendas.', 'Adolfo Suarez Madrid-Barajas Airport is located in Madrid, Paracuellos de Jarama, San Sebastian de los Reyes and Alcobendas.']","Adolfo Suarez Madrid-Barajas Airport is located in Madrid, Paracuellos de Jarama, San Sebastian de los Reyes and Alcobendas."
"['The runway name of Adolfo Suarez Madrid-Barajas Airport is 18L/36R.', 'The runway name at Adolfo Suarez Madrid-Barajas airport is 18L/36R.', 'The runway name of Adolfo Suarez Madrid-Barajas Airport is 18L/36R.']",The runway name of Adolfo Suarez Madrid-Barajas Airport is 18L/36R.
"['Afonso Pena International Airport ICAO Location Idenitifier is SBCT.', 'SBCT is the ICAO location identifier of Afonso Pena International Airport.']",The ICAO location identifier of Afonso Pena International Airport is SBCT.
"['Afonso Pena International Airport serves the city of Curitiba.', 'Afonso Pena International Airport serves Curitiba.']",Afonso Pena International Airport serves the city of Curitiba.
"['The Al Taqaddum Air Base serves the city of Fallujah.', 'Al-Taqaddum Air Base serves the city of Fallujah.']",Al-Taqaddum Air Base serves the city of Falujah.
"['The runway length of Al-Taqaddum Air Base is 3684.0.', 'The length of the runway at Al-Taqaddum Air Base is 3684.0.']",The runway length of Al-Taqaddum Air Base is 3684.0.
"['Alderney Airport runway name is 14/32.', '14/32 is the runway name of Alderney Airport.', 'The runway name of Alderney Airport is 14/32.']",The runway name of Alderney Airport is 14/32.
"['The runway length at Allama Iqbal International Airport is 3,360.12.', 'The runway at Allama Iqbal International Airport has a length of 3360.12.', 'The runway at Allama Iqbal International Airport is 3360.12 long.']",The runway length of Allama Iqbal International Airport is 3360.12.
"[""The first runway at Amsterdam's Schiphol Airport is known as Number 18."", ""The Amsterdam Airport Schiphol's 1st runway number is 18."", 'The number of the 1st runway at Amsterdam Airport Schiphol is 18.']",The 1st runway number at Amsterdam Airport Schiphol is 18.


In [8]:
rouge_scores = (
  chunked.starmap(compute_rouge)
    # only one rouge per sample, so take the first of the values
    .map(lambda x: seq(x.items()).starmap(lambda x,y : {x:y[0]}))
    # rolling union on dictionaries since they are disjoint
    .map(lambda x: seq(x).reduce(lambda x, y: x | y))
    .to_pandas()
)
rouge_scores

,rouge1,rouge2,rougeL,rougeLsum
0,0.947368,0.823529,0.947368,0.947368
1,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,1.000000
3,1.000000,0.800000,0.818182,0.818182
4,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...
1595,0.888889,0.693878,0.646465,0.646465
1596,0.650000,0.379747,0.493827,0.493827
1597,0.896552,0.776471,0.827586,0.827586
1598,0.800000,0.574713,0.700000,0.700000


In [9]:
rouge_scores.describe()

,rouge1,rouge2,rougeL,rougeLsum
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,0.848860,0.672264,0.745394,0.745394
std,0.110731,0.200575,0.177137,0.177137
min,0.387097,0.125000,0.266667,0.266667
25%,0.780488,0.533333,0.612245,0.612245
50%,0.851064,0.666667,0.750000,0.750000
75%,0.933871,0.800000,0.894737,0.894737
max,1.000000,1.000000,1.000000,1.000000


In [10]:
bleu = load('sacrebleu')
bleu
compute_bleu = lambda x,y: bleu.compute(references=[x], predictions=[y])

In [11]:
bleu_scores = (
  chunked.starmap(compute_bleu)
    .to_pandas()
)

In [12]:
bleu_scores

,score,counts,totals,precisions,bp,sys_len,ref_len
0,83.499502,"[11, 10, 7, 5]","[11, 10, 9, 8]","[100.0, 100.0, 77.77777777777777, 62.5]",1.000000,11,11
1,100.000000,"[21, 20, 19, 18]","[21, 20, 19, 18]","[100.0, 100.0, 100.0, 100.0]",1.000000,21,21
2,100.000000,"[13, 12, 11, 10]","[13, 12, 11, 10]","[100.0, 100.0, 100.0, 100.0]",1.000000,13,13
3,73.488892,"[11, 9, 7, 5]","[12, 11, 10, 9]","[91.66666666666667, 81.81818181818181, 70.0, 5...",1.000000,12,12
4,100.000000,"[10, 9, 8, 7]","[10, 9, 8, 7]","[100.0, 100.0, 100.0, 100.0]",1.000000,10,10
...,...,...,...,...,...,...,...
1595,75.985407,"[51, 44, 38, 32]","[55, 54, 53, 52]","[92.72727272727273, 81.48148148148148, 71.6981...",1.000000,55,52
1596,26.701496,"[29, 19, 12, 8]","[34, 33, 32, 31]","[85.29411764705883, 57.57575757575758, 37.5, 2...",0.571881,34,53
1597,76.634498,"[43, 37, 32, 29]","[47, 46, 45, 44]","[91.48936170212765, 80.43478260869566, 71.1111...",1.000000,47,47
1598,52.649087,"[47, 35, 25, 19]","[57, 56, 55, 54]","[82.45614035087719, 62.5, 45.45454545454545, 3...",0.982609,57,58


In [13]:
# TODO: bertscore next, then save all evaluation and predicition
bertscore = load('bertscore')
compute_bert = lambda x,y: bertscore.compute(predictions=[y], references=[x], lang="en", model_type="distilbert-base-uncased" )

In [14]:
bert_scores = (
  chunked
 .starmap(compute_bert)
 .to_pandas()
 .drop(columns='hashcode')
 .applymap(np.mean)
)

In [15]:
prepend_name_to_cols = lambda x,y : x.rename(columns=lambda e: y+"_"+e)
prepend_name_to_cols(bert_scores, 'bert')
all_scores = (
  seq(bert_scores, bleu_scores, rouge_scores)
    .zip(['bert','bleu','rouge'])
    .starmap(prepend_name_to_cols)
)
scores_df = pd.concat(all_scores, axis=1)
scores_df

,bert_precision,bert_recall,bert_f1,bleu_score,bleu_counts,bleu_totals,bleu_precisions,bleu_bp,bleu_sys_len,bleu_ref_len,rouge_rouge1,rouge_rouge2,rouge_rougeL,rouge_rougeLsum
0,0.988835,0.977650,0.983211,83.499502,"[11, 10, 7, 5]","[11, 10, 9, 8]","[100.0, 100.0, 77.77777777777777, 62.5]",1.000000,11,11,0.947368,0.823529,0.947368,0.947368
1,1.000000,1.000000,1.000000,100.000000,"[21, 20, 19, 18]","[21, 20, 19, 18]","[100.0, 100.0, 100.0, 100.0]",1.000000,21,21,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,100.000000,"[13, 12, 11, 10]","[13, 12, 11, 10]","[100.0, 100.0, 100.0, 100.0]",1.000000,13,13,1.000000,1.000000,1.000000,1.000000
3,0.967089,0.967089,0.967089,73.488892,"[11, 9, 7, 5]","[12, 11, 10, 9]","[91.66666666666667, 81.81818181818181, 70.0, 5...",1.000000,12,12,1.000000,0.800000,0.818182,0.818182
4,1.000000,1.000000,1.000000,100.000000,"[10, 9, 8, 7]","[10, 9, 8, 7]","[100.0, 100.0, 100.0, 100.0]",1.000000,10,10,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0.936552,0.932994,0.933446,75.985407,"[51, 44, 38, 32]","[55, 54, 53, 52]","[92.72727272727273, 81.48148148148148, 71.6981...",1.000000,55,52,0.888889,0.693878,0.646465,0.646465
1596,0.926603,0.883873,0.904734,26.701496,"[29, 19, 12, 8]","[34, 33, 32, 31]","[85.29411764705883, 57.57575757575758, 37.5, 2...",0.571881,34,53,0.650000,0.379747,0.493827,0.493827
1597,0.981708,0.983372,0.982539,76.634498,"[43, 37, 32, 29]","[47, 46, 45, 44]","[91.48936170212765, 80.43478260869566, 71.1111...",1.000000,47,47,0.896552,0.776471,0.827586,0.827586
1598,0.936510,0.950141,0.933748,52.649087,"[47, 35, 25, 19]","[57, 56, 55, 54]","[82.45614035087719, 62.5, 45.45454545454545, 3...",0.982609,57,58,0.800000,0.574713,0.700000,0.700000


In [16]:
scores_df.describe()

,bert_precision,bert_recall,bert_f1,bleu_score,bleu_bp,bleu_sys_len,bleu_ref_len,rouge_rouge1,rouge_rouge2,rouge_rougeL,rouge_rougeLsum
count,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000
mean,0.956341,0.950560,0.952563,62.730283,0.962085,21.970000,22.190000,0.848860,0.672264,0.745394,0.745394
std,0.031023,0.034165,0.031925,23.534629,0.077608,10.917467,10.943148,0.110731,0.200575,0.177137,0.177137
min,0.815483,0.802319,0.810992,5.346660,0.296922,5.000000,5.000000,0.387097,0.125000,0.266667,0.266667
25%,0.935498,0.927892,0.930835,46.253942,0.953497,13.000000,13.000000,0.780488,0.533333,0.612245,0.612245
50%,0.957584,0.951502,0.953139,62.177741,1.000000,21.000000,21.000000,0.851064,0.666667,0.750000,0.750000
75%,0.980980,0.976880,0.977398,78.851324,1.000000,29.000000,29.000000,0.933871,0.800000,0.894737,0.894737
max,1.000000,1.000000,1.000000,100.000000,1.000000,81.000000,67.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
scores_df.to_pickle(OUTPUT_PATH/"d2s_scores.pkl")